In [ ]:
from huggingface_hub import hf_hub_download, snapshot_download
repo_path = snapshot_download(repo_id="meta-llama/Llama-3.2-3B")


Fetching 15 files:   0%|          | 0/15 [00:00<?, ?it/s]Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`
Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`
c:\Users\khatt\Documents\AmazonEcho-NLP-Project\venv\lib\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\khatt\.cache\huggingface\hub\models--meta-llama--Llama-3.2-3B. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_H

In [1]:
print("HelloWorld")

HelloWorld


In [122]:
import os
from dotenv import load_dotenv
load_dotenv()
model_name = "E:\\models\\pleasRAG"
token = os.getenv("HFTOKEN")
import torch

In [123]:
from transformers import AutoModelForCausalLM 
from transformers import AutoTokenizer
model = AutoModelForCausalLM.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)

In [140]:
from transformers import pipeline
pipe = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    torch_dtype=torch.bfloat16,
    device_map="auto",
    return_full_text=True
)

Device set to use cpu


In [125]:
pipe("The key to life is")

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


[{'generated_text': 'The key to life is love. (Matthew 20:22)\n\nThe question of whether or not love is a good thing, a good principle, is one that needs to be wrestled with. The most that we can say about love is that it is the most important of the many basic principles. Love is the most basic of all human principles. In the words of St. Paul, love is the love of our neighbors.\n3 [I was] a man of the world and, above all, a man of action. I knew nothing of the world, and no one knew me. I became one of the world’s most remarkable men.\n[C]haracter:\n\nI became the second man of the world, I was the second man of action. I became an Italian in the world, and a man of the world in action.\n[D]eath:\n\nThe first day of the year I was murdered. I was murdered by the death of a handful of Italian fascists who wanted to stop me from becoming the second man of the world.\n[E]xile:\n\nI lived in exile for a year.\n[F]ascism:\n\nI was fascist for'}]

In [61]:
import pandas as pd
df = pd.read_csv("Amazon Echo 2 Reviews.csv")

In [62]:
df = df.head(30)

In [ ]:
from langchain_community.embeddings import HuggingFaceEmbeddings
model_name = "E:/models/gte-small"


In [ ]:
embedding_model = HuggingFaceEmbeddings(
    model_name=model_name,
    multi_process=True,
    model_kwargs={"device": "cpu"},
    encode_kwargs={"normalize_embeddings": True},  # Set `True` for cosine similarity
    
)


C:\Users\khatt\AppData\Local\Temp\ipykernel_22340\2104233462.py:1: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embedding_model = HuggingFaceEmbeddings(


In [7]:
from dotenv import load_dotenv
import os
load_dotenv()
PINECONE = os.getenv("PINECONE")
print(PINECONE)

pcsk_5v561g_8Y3ZthdWE8zpveZ9s6Mm4RUVH7Rnyxm6NxRJgDVdnJKGdf5khDtYhxhhA49rPdR


In [ ]:
vectorized_input = embedding_model.embed_query("This is a test")

In [ ]:
from pinecone import Pinecone
pc = Pinecone(api_key=PINECONE)
index = pc.Index("Echo")

In [63]:
data = []
for i in range(len(df)):
    text = df['Review Text'][i]
    vector = embedding_model.embed_query(text)
    data.append(
        {
        "id":"vec{}".format(i),
        "values":vector,
        "metadata":{"text":text}
    })

In [65]:
index.upsert(
    vectors=data,
    namespace= "echo"
)

KeyboardInterrupt: 

In [133]:
prompt = """
<|system|>
You are an AI helpful assistant that answers on questions based on the reviews of the people from the source and in the context.
Give the rational and well written response. If you don't have proper info in the context, answer "I don't know"
Respond only to the question asked.

<|user|>
Context:
{context}
---Here is the question you need to answer.

Question: {question}
<|assistant|>


"""

In [ ]:
user_input = input("User: ")

vectorized_input = embedding_model.embed_query(user_input)

context = index.query(
    namespace="echo",
    vector=vectorized_input,
    top_k=2,
    include_metadata=True
)


In [134]:
combinedcontext = [context['matches'][i]['metadata']['text'] for i in range(len(context['matches']))]
combinedcontext=str(".".join(combinedcontext))
combinedcontext

'Awesome product! However, it does take time for Alexa to recognize other language when asked to play Spanish music and after a while she’ll turn off by it self..I like it very much so. Especially when I am typing a letter or email and my mind goes blank on spelling some words.  Alexa will spell them for you as you type.  Plays my Pandora music constantly, gets news updates, weather and most trivial questions.'

In [141]:
answer = pipe(prompt.format(context = combinedcontext, question = user_input))


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


In [142]:
answer

[{'generated_text': '\n<|system|>\nYou are an AI helpful assistant that answers on questions based on the reviews of the people from the source and in the context.\nGive the rational and well written response. If you don\'t have proper info in the context, answer "I don\'t know"\nRespond only to the question asked.\n\n<|user|>\nContext:\nAwesome product! However, it does take time for Alexa to recognize other language when asked to play Spanish music and after a while she’ll turn off by it self..I like it very much so. Especially when I am typing a letter or email and my mind goes blank on spelling some words.  Alexa will spell them for you as you type.  Plays my Pandora music constantly, gets news updates, weather and most trivial questions.\n---Here is the question you need to answer.\n\nQuestion: do people like the product?\n<|assistant|>\n\n\nRescued by the context, I can provide a response based on the available information. The product is indeed a great one. Alexa will do her bes

In [143]:
print(answer[0]['generated_text'])


<|system|>
You are an AI helpful assistant that answers on questions based on the reviews of the people from the source and in the context.
Give the rational and well written response. If you don't have proper info in the context, answer "I don't know"
Respond only to the question asked.

<|user|>
Context:
Awesome product! However, it does take time for Alexa to recognize other language when asked to play Spanish music and after a while she’ll turn off by it self..I like it very much so. Especially when I am typing a letter or email and my mind goes blank on spelling some words.  Alexa will spell them for you as you type.  Plays my Pandora music constantly, gets news updates, weather and most trivial questions.
---Here is the question you need to answer.

Question: do people like the product?
<|assistant|>


Rescued by the context, I can provide a response based on the available information. The product is indeed a great one. Alexa will do her best to process this query as a Spanish l

In [9]:
import requests

response = requests.post(
    "http://127.0.0.1:8000/extract-data",
    json={"url": "https://www.amazon.com/Amazon-Basics-Everyday-Plates-Disposable/dp/B0C2CY22B8/?_encoding=UTF8&pd_rd_w=APjaP&content-id=amzn1.sym.f2128ffe-3407-4a64-95b5-696504f68ca1&pf_rd_p=f2128ffe-3407-4a64-95b5-696504f68ca1&pf_rd_r=YXMEXN435CNCQZWD512A&pd_rd_wg=7Izqy&pd_rd_r=3fbb0d02-8f22-4b11-ac45-f11f9ef282d6&ref_=pd_hp_d_btf_crs_zg_bs_284507&th=1"}
)

data = response.json()
print(data)

{'columns': ['Unnamed: 0', 'AmazonBasicsEverydayPlatesDisposable'], 'rows': [{'Unnamed: 0': 0, 'AmazonBasicsEverydayPlatesDisposable': "\nThese came on time with my other items. They are durable, leak proof, hold a decent serving size, and great for any occasion. The coloring is neutral, if it matters to you. Great price for that quantity. I'll definitely keep buying them here.\n"}, {'Unnamed: 0': 1, 'AmazonBasicsEverydayPlatesDisposable': '\nWe go through lots of paper plates at my house.  These are of great quality and have saved me a lot of time and money by putting them on a subscribe and save option.  I love that they are eco-friendly and they are the perfect size for any meal.  All you mammas out there - gift yourself some time from washing those extra dishes and go play with those kiddos!\n'}, {'Unnamed: 0': 2, 'AmazonBasicsEverydayPlatesDisposable': '\n✅ Durability & Performance:- Thicker than expected, handling heavy meals without bending.- Solid grease resistance, keeping han

In [39]:
import requests

payload = {
    "url": "https://www.amazon.com/Amazon-Basics-Everyday-Plates-Disposable/dp/B0C2CY22B8/?_encoding=UTF8&pd_rd_w=APjaP&content-id=amzn1.sym.f2128ffe-3407-4a64-95b5-696504f68ca1&pf_rd_p=f2128ffe-3407-4a64-95b5-696504f68ca1&pf_rd_r=YXMEXN435CNCQZWD512A&pd_rd_wg=7Izqy&pd_rd_r=3fbb0d02-8f22-4b11-ac45-f11f9ef282d6&ref_=pd_hp_d_btf_crs_zg_bs_284507&th=1",
    "question": "tell me about the product"
}

try:
    response = requests.post("http://127.0.0.1:8000/ask", json=payload)

    print("📦 Raw response:")
    # print(response.text)  # <-- This shows you exactly what the server returned

    response.raise_for_status()  # Raises an error for 4xx/5xx responses

    data = response.json()
    print("✅ Parsed JSON:", data)

except requests.exceptions.HTTPError as err:
    print(f"❌ HTTP error: {err}")
except requests.exceptions.RequestException as err:
    print(f"❌ Request failed: {err}")
except ValueError as err:
    print(f"❌ JSON decode error: {err}")


📦 Raw response:
✅ Parsed JSON: {'answer': 'Based on the reviews, this product is a set of disposable plates that excel in durability and performance. They are thicker than expected, making them suitable for handling heavy meals without bending or breaking. The plates also feature solid grease resistance, keeping hands clean even when dealing with saucy dishes. The plates come in a perfect size for everyday use, making them ideal for casual meals and gatherings. While not as rigid as premium brands, they are a great value for the price, offering reliable performance for affordable disposable plates.'}


In [38]:
import requests 
response = requests.post(
    "http://127.0.0.1:8000/ask",
    json={
    "url":"https://www.amazon.com/Amazon-Basics-Everyday-Plates-Disposable/dp/B0C2CY22B8/?_encoding=UTF8&pd_rd_w=APjaP&content-id=amzn1.sym.f2128ffe-3407-4a64-95b5-696504f68ca1&pf_rd_p=f2128ffe-3407-4a64-95b5-696504f68ca1&pf_rd_r=YXMEXN435CNCQZWD512A&pd_rd_wg=7Izqy&pd_rd_r=3fbb0d02-8f22-4b11-ac45-f11f9ef282d6&ref_=pd_hp_d_btf_crs_zg_bs_284507&th=1",
    "quesiton":"What do you think about the product"
    }
)
data = response.json()
print(data)

JSONDecodeError: Expecting value: line 1 column 1 (char 0)

In [6]:
import requests 
response = requests.post(
    "http://127.0.0.1:8000/ans",
    json={
    "url":"https://www.amazon.com/Amazon-Basics-Everyday-Plates-Disposable/dp/B0C2CY22B8/?_encoding=UTF8&pd_rd_w=APjaP&content-id=amzn1.sym.f2128ffe-3407-4a64-95b5-696504f68ca1&pf_rd_p=f2128ffe-3407-4a64-95b5-696504f68ca1&pf_rd_r=YXMEXN435CNCQZWD512A&pd_rd_wg=7Izqy&pd_rd_r=3fbb0d02-8f22-4b11-ac45-f11f9ef282d6&ref_=pd_hp_d_btf_crs_zg_bs_284507&th=1",
    "question":"What do you think about the product"
    }
)
data = response.json()
print(data)

ConnectionError: HTTPConnectionPool(host='127.0.0.1', port=8000): Max retries exceeded with url: /ans (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x0000022EE298F250>: Failed to establish a new connection: [WinError 10061] No connection could be made because the target machine actively refused it'))

In [8]:
import requests 
response = requests.post(
    "http://127.0.0.1:8000/review",
    json={
    "url":"https://www.amazon.com/Amazon-Basics-Everyday-Plates-Disposable/dp/B0C2CY22B8/?_encoding=UTF8&pd_rd_w=APjaP&content-id=amzn1.sym.f2128ffe-3407-4a64-95b5-696504f68ca1&pf_rd_p=f2128ffe-3407-4a64-95b5-696504f68ca1&pf_rd_r=YXMEXN435CNCQZWD512A&pd_rd_wg=7Izqy&pd_rd_r=3fbb0d02-8f22-4b11-ac45-f11f9ef282d6&ref_=pd_hp_d_btf_crs_zg_bs_284507&th=1"
    }
)
data = response.json()
print(data)

JSONDecodeError: Expecting value: line 1 column 1 (char 0)

In [3]:
import requests 
response = requests.post(
    "https://amazonecho-nlp-project.onrender.com/ans",
    json={
    "url":"https://www.amazon.com/Amazon-Basics-Everyday-Plates-Disposable/dp/B0C2CY22B8/?_encoding=UTF8&pd_rd_w=APjaP&content-id=amzn1.sym.f2128ffe-3407-4a64-95b5-696504f68ca1&pf_rd_p=f2128ffe-3407-4a64-95b5-696504f68ca1&pf_rd_r=YXMEXN435CNCQZWD512A&pd_rd_wg=7Izqy&pd_rd_r=3fbb0d02-8f22-4b11-ac45-f11f9ef282d6&ref_=pd_hp_d_btf_crs_zg_bs_284507&th=1",
    "question":"What do you think about the product"
    }
)
data = response.json()
print(data)

{'answer': "Based on the reviews, I think this product is a great value for its price. It's durable, grease-resistant, and perfect for everyday use. The reviewer who works full-time and has kids who dislike doing dishes seems to really appreciate its convenience. The fact that it's strong and can handle heavy meals without bending is also a major plus. Overall, it seems like a solid choice for anyone looking for affordable, reliable disposable plates."}


In [ ]:
response = requests.post(
    "http://127.0.0.1:8000/review",  # 👈 Local URL
    json={
        "url": "https://www.amazon.com/Amazon-Basics-Everyday-Plates-Disposable/dp/B0C2CY22B8/?_encoding=UTF8&pd_rd_w=APjaP&content-id=amzn1.sym.f2128ffe-3407-4a64-95b5-696504f68ca1&pf_rd_p=f2128ffe-3407-4a64-95b5-696504f68ca1&pf_rd_r=YXMEXN435CNCQZWD512A&pd_rd_wg=7Izqy&pd_rd_r=3fbb0d02-8f22-4b11-ac45-f11f9ef282d6&ref_=pd_hp_d_btf_crs_zg_bs_284507&th=1"
    }
)

print(response.status_code)
print(response.text)

200
{"columns":["Unnamed: 0","AmazonBasicsEverydayPlatesDisposable"],"rows":[{"Unnamed: 0":0,"AmazonBasicsEverydayPlatesDisposable":"\nThese came on time with my other items. They are durable, leak proof, hold a decent serving size, and great for any occasion. The coloring is neutral, if it matters to you. Great price for that quantity. I'll definitely keep buying them here.\n"},{"Unnamed: 0":1,"AmazonBasicsEverydayPlatesDisposable":"\nWe go through lots of paper plates at my house.  These are of great quality and have saved me a lot of time and money by putting them on a subscribe and save option.  I love that they are eco-friendly and they are the perfect size for any meal.  All you mammas out there - gift yourself some time from washing those extra dishes and go play with those kiddos!\n"},{"Unnamed: 0":2,"AmazonBasicsEverydayPlatesDisposable":"\n✅ Durability & Performance:- Thicker than expected, handling heavy meals without bending.- Solid grease resistance, keeping hands clean ev

In [32]:
data

'Internal Server Error'

In [24]:
from gradio_client import Client
client = Client("UmarKhattab09/AmazonNLPproject", verbose=True)

print(client.view_api())  # Shows function names, inputs, etc.

Loaded as API: https://umarkhattab09-amazonnlpproject.hf.space ✔
Client.predict() Usage Info
---------------------------
Named API endpoints: 2

 - predict(url, api_name="/dfreviews") -> value_8
    Parameters:
     - [Textbox] url: str (required)  
    Returns:
     - [Dataframe] value_8: dict(headers: list[Any], data: list[list[Any]], metadata: dict(str, list[Any] | None) | None) 

 - predict(url, question, api_name="/getans") -> answer
    Parameters:
     - [Textbox] url: str (required)  
     - [Textbox] question: str (required)  
    Returns:
     - [Textbox] answer: str 

None
